In [1]:
using NBInclude
@nbinclude("dcj_algo.ipynb")

In [ ]:
# returns the index of the following array 
# a b c ... x y z aa ab ac ... ax ay az ba bb bc ... bx by bz ... ca cb cd ....
# (doesn't build the array)

function first_n_letters_no_limits(n::Int)
    alphabet = 'a':'z'  # Define the alphabet range
    letters = String[]
    
    for i in 1:n
        idx = i - 1
        letter = ""
        
    while idx >= 0
            place_value = idx % 26 + 1
            letter = string(alphabet[place_value], letter)
            idx = idx ÷ 26 - 1
        end
        
        push!(letters, letter)
    end
    
    return letters
end

# letters = first_n_letters_no_limits(27)


In [3]:
# generates target genomes that contain the first n letters and adds x duplicate letters 
# x >= 1
function generate_target_with_xdup(n::Int, x::Int)
    # target 
    alphabet = first_n_letters_no_limits(n)
    genome = copy(alphabet)
    
    for i in 1:x
        rand_dup_gene = alphabet[rand(1:length(alphabet))]
        rand_idx = rand(1:length(genome)+1)
        insert!(genome, rand_idx, rand_dup_gene)
    end 

    return join(genome, "'")  # one chrom 
end 

# target = generate_target_with_xdup(5, 2)
# print(target)

generate_target_with_xdup (generic function with 1 method)

In [4]:
function add_gene_counts(input_string::String)  
    genes = split(input_string, "'")

    gene_count = Dict{String, Int}()
    result = IOBuffer()

    for gene in genes
        if gene == '.' || gene == ',' 
            print(result, gene) 
            continue 
        end 
        if !haskey(gene_count, gene)
            gene_count[gene] = 0
        end
        gene_count[gene] += 1
        print(result, gene, gene_count[gene])
    end

    return String(take!(result))
end

# input_string = "he'll'o"
# result = add_letter_counts(input_string)
# println(result)  # Output: "he1ll1o1"

add_gene_counts (generic function with 1 method)

In [5]:
tar = generate_target_with_xdup(27, 3)
tar_wcounts = add_gene_counts(tar)
println(tar) 
println(tar_wcounts)

a'b'c'd'e'f'g'h'i'j'k'l'm'n'o'p'q'r'v's't'u'i'v'w'x'y'z'a'aa
a1b1c1d1e1f1g1h1i1j1k1l1m1n1o1p1q1r1v1s1t1u1i2v2w1x1y1z1a2aa1


In [35]:
function apply_dcjop(adj_list::Vector{Adjacency}, gid_to_loc::DefaultDict{Int, Vector{Int}}, m::Int)
    randidx1 = rand(1:length(adj_list))
    randidx2 = rand(1:length(adj_list))
    while randidx1 == randidx2 
        randidx2 = rand(1:length(adj_list))
    end

    adj_to_change1 = adj_list[randidx1]
    adj_to_change2 = adj_list[randidx2]

    adj1_swapleft = rand(Bool)
    adj2_swapleft = rand(Bool)

    if adj1_swapleft 
        adj1_ge = adj_list[randidx1].left 
    else 
        adj1_ge = adj_list[randidx1].right 
    end 
    if adj2_swapleft 
        adj2_ge = adj_list[randidx2].left 
    else 
        adj2_ge = adj_list[randidx2].right 
    end 
    
    new_adj1, new_adj2 = update_documentation(adj1_ge, adj2_ge, randidx1, randidx2, adj_list, gid_to_loc)

    if m >= 1
        print("DCJ :: ")           
        print("[idxs ", randidx1, " ", randidx2, "] ")         
        show(adj_to_change1, true) 
        show(adj_to_change2, true)

        print(" --> ")
        
        show(new_adj1, true) 
        show(new_adj2, true)
        println()
    end 
end 

function apply_x_dcjops(target::String, x::Int, m::Int)
    # turn target genome into list of adjacencies
    id_counter = Ref{Int}(1)
    id_to_str = Dict{Int, String}()
    str_to_id = Dict{String, Int}()

    target_genome = string_to_genome(target, id_counter, id_to_str, str_to_id, true)
    adj_list = genome_to_adj_list(target_genome)  
    gid_to_loc = process_adj_list(adj_list)   
    
    if m >= 1
        show(adj_list)
        println()
    end 
    
    for i in 1:x
        apply_dcjop(adj_list, gid_to_loc, m)
    end 
    
    # convert adj back to genome 

    #  TODO 
    # inorder_adj_list = reorder_adjs(src_adjs, src_gid_to_l)
    # src_genome_str = adj_list_to_genome_str(inorder_adj_list)
   
    # print(src_genome_str)

    if m >= 1
        show(adj_list)
        println()
    end 

    # return src_genome_str
end 


apply_x_dcjops(tar_wcounts, 3, 1)

(a1:h,b1:t)(b1:h,c1:t)(c1:h,d1:t)(d1:h,e1:t)(e1:h,f1:t)(f1:h,g1:t)(g1:h,h1:t)(h1:h,i1:t)(i1:h,j1:t)(j1:h,k1:t)(k1:h,l1:t)(l1:h,m1:t)(m1:h,n1:t)(n1:h,o1:t)(o1:h,p1:t)(p1:h,q1:t)(q1:h,r1:t)(r1:h,v1:t)(v1:h,s1:t)(s1:h,t1:t)(t1:h,u1:t)(u1:h,i2:t)(i2:h,v2:t)(v2:h,w1:t)(w1:h,x1:t)(x1:h,y1:t)(y1:h,z1:t)(z1:h,a2:t)(a2:h,aa1:t)(aa1:h,a1:t)
DCJ :: [idxs 11 5] (k1:h,l1:t)(e1:h,f1:t) --> (l1:t,f1:t)(k1:h,e1:h)
DCJ :: [idxs 8 27] (h1:h,i1:t)(y1:h,z1:t) --> (i1:t,y1:h)(h1:h,z1:t)
DCJ :: [idxs 23 17] (i2:h,v2:t)(q1:h,r1:t) --> (v2:t,q1:h)(i2:h,r1:t)
(a1:h,b1:t)(b1:h,c1:t)(c1:h,d1:t)(d1:h,e1:t)(k1:h,e1:h)(f1:h,g1:t)(g1:h,h1:t)(i1:t,y1:h)(i1:h,j1:t)(j1:h,k1:t)(l1:t,f1:t)(l1:h,m1:t)(m1:h,n1:t)(n1:h,o1:t)(o1:h,p1:t)(p1:h,q1:t)(i2:h,r1:t)(r1:h,v1:t)(v1:h,s1:t)(s1:h,t1:t)(t1:h,u1:t)(u1:h,i2:t)(v2:t,q1:h)(v2:h,w1:t)(w1:h,x1:t)(x1:h,y1:t)(h1:h,z1:t)(z1:h,a2:t)(a2:h,aa1:t)(aa1:h,a1:t)
